[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/okada-tak/deep-learning-from-scratch-2/blob/master/notebooks/ch04.ipynb)

# 4章 word2vecの高速化 のまとめ
- Embeddedレイヤは単語の分散表現を格納し、順伝播において該当する単語IDのベクトルを抽出する
- word2vecでは語彙数の増加に比例して計算量が増加するので、近似計算を行う高速な手法を使うとよい
- Negative Samplingは負例をいくつかサンプリングする手法であり、これを利用すれば多値分類を二値分類として扱うことができる
- word2vecによって得られた単語の分散表現は、単語の意味が埋め込まれたものであり、似たコンテキストで使われる単語は単語ベクトルの空間上で近い場所に位置するようになる
- word2vecの単語の分散表現は、類推問題をベクトルの加算と減算によって解ける性質を持つ
- word2vecは転移学習の点で特に重要であり、その単語の分散表現はさまざまな自然言語処理のタスクに利用できる

## 4.1 word2vecの改良①
ボトルネック：  
- 入力層のone-hot表現と重み行列$\text{W}_\text{in}$の積による計算（4.1節で解決）
- 中間層と重み行列$\text{W}_\text{out}$の積およびSoftmaxレイヤの計算（4.2節で解決）



### 4.1.1 Embeddingレイヤ
重みパラメータから「単語IDに該当する行（ベクトル）」を抜き出すレイヤ：Embdddingレイヤ  
単語の分散表現が格納される

### 4.1.2 Embeddingレイヤの実装


In [1]:
import numpy as np
W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
W[2]

array([6, 7, 8])

In [4]:
W[5]

array([15, 16, 17])

In [5]:
idx = np.array([1,0,3,0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

common/layers.py

In [6]:
# p.136-137
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

# p.138
#    def backward(self, dout):
#        dW, = self.grads
#        dW[...] = 0
#        dW[self.idx] = dout     # 悪い例
#        return None

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
#        if GPU:
#            np.scatter_add(dW, self.idx, dout)
#        else:
        np.add.at(dW, self.idx, dout)
        return None

## 4.2 word2vecの改良②
Negative Samplingを導入する

### 4.2.1 中間層以降の計算の問題点